# C_L_US-CA_ConfirmedCases

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque
import time

import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version

import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from platform import python_version
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from datetime import datetime
import pickle
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import RandomOverSampler


# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 200)

In [ ]:
#'.[桁数]e'
pd.options.display.float_format = '{:10.1f}'.format

In [ ]:
print(pd.options.display.float_format)

# definition

In [ ]:
### customize define

FEATURE_NUM = 90

# 特徴量削減ステップ
SELECTOR_STEP = .05

#TARGET_PROB_CLASS_VALUE = '1'
TARGET_CLASS_VALUE = 1

CV_VAL = 5

# n_estimators:ここを増やすと精度は上がるが遅い。。。
#ESTIMATOR_NUM=5
ESTIMATOR_NUM=10
#ESTIMATOR_NUM=30
#ESTIMATOR_NUM=100


# 特徴量選択のアルゴリズムタイプ
# RandomForest
FEATURE_SELECT_TYPE_RANDOM_FOREST = 1
# Logistic
FEATURE_SELECT_TYPE_LOGISTIC= 2

# 特長量選択に実際に使うアルゴリズムタイプ
feature_select_type = FEATURE_SELECT_TYPE_LOGISTIC



In [ ]:
# カラム名変更フラグ
COL_NAME_CHG_FLG = True

read_train_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/reg/COVID19 Local US-CA/input/ca_train.csv"

read_test_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/reg/COVID19 Local US-CA/input/ca_test.csv"


output_dir_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/reg/COVID19 Local US-CA/output"

pkl_dir_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/reg/COVID19 Local US-CA/pkl"

output_file_name = "submission.csv"

drop_col_name_list =[
    "Province/State", "Country/Region", "Lat", "Long"
]

float_conv_col_name_list = [
    "Date"
]

float_nan_conv_dic ={
    # name: [min, max]
}

# その他独自変換処理用カラムリスト
other_conv_process = [
    "Date"
]


id_col_name="ForecastId"
target_col_name = "ConfirmedCases"

RANDOM_STATE_VAL = 0


# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
#     print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
#     for col_name in readDf.columns.tolist():
#         print("{} : {}".format(col_name, readDf[col_name].nunique()))
    

    

In [ ]:
def chkNum(expect_label, expect_num, act_label, act_num):
    if expect_num != act_num:
        raise Exception("Num Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_num, act_label, act_num))
    

In [ ]:
def standardScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # 平準化
    scaler = StandardScaler()
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkNum("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkNum("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkNum("dataLen", dataLen, "retDf", len(retDf))

    return retDf

In [ ]:
# 予測
def getPredictDf(model, test_Df, id_Df , target_col_name):
    return id_Df.join(
            pd.DataFrame(model.predict(test_Df),
                         dtype=np.int, columns=[target_col_name]))

In [ ]:
# その他変換処理を行う。
def  otherConvProcess(convDf, other_conv_process):
    dateime_col_name = other_conv_process[0]
    date_val_list = convDf[dateime_col_name].values.tolist()
    conv_val_list = [datetime.strptime(elem, '%Y-%m-%d').timestamp() for elem in date_val_list]
    convDf[dateime_col_name] = pd.Series(conv_val_list)
    dispalyDf(convDf, True, 5,  "convDf")

    return convDf

In [ ]:
# 前処理関数
def getBeforeProcessDf(inputDf, drop_col_name_list, float_conv_col_name_list, float_nan_conv_dic,
                       train_Flg, train_target_Df ,
                       random_state_val, feature_num, 
                       feature_select_type, 
                       selector_step, train_col_list, other_conv_process):
    # inputをコピーしておく
    convDf = inputDf.copy()
    
    # 列削除
    convDf = convDf.drop(drop_col_name_list, axis=1)
    
    # カラム名リスト保持
    col_list = convDf.columns.tolist()
    
    # その他変換処理
    convDf = otherConvProcess(convDf, other_conv_process)
    

    print("連続値のカラムの欠損値補完 = ",datetime.now())
    # 連続値のカラムの欠損値補完
    float_conv_dic={}
    for col_name in float_conv_col_name_list:
        float_values_list = [np.float(elem)  for elem in convDf[col_name].values.tolist()]
        
        if len(float_nan_conv_dic) > 0:
            conv_list = \
            [(
                np.random.randint(
                    float_nan_conv_dic[col_name][0],
                    float_nan_conv_dic[col_name][1])
                if col_name in float_nan_conv_dic else 0
            if np.isnan(elem) else elem )
             for elem in float_values_list]
            
            float_conv_dic[col_name]=conv_list
            del conv_list
        else:
            float_conv_dic[col_name]=float_values_list
        
    print("dummy = ",datetime.now())
    ##### float変換がある場合 ####
    if len(float_conv_dic)>0:
        # DataFrameを起こす。
        float_conv_df = pd.DataFrame(float_conv_dic,dtype=np.float)
    
        #dispalyDf(float_conv_df, True, 5,  "float_conv_df")

        col_list_except_float = [col_name for col_name in col_list 
                                 if col_name not in float_conv_col_name_list]

        # float以外のカラムが存在する場合
        if len(col_list_except_float) > 0:
            # float以外のDataFrameにする
            convDf = convDf.loc[:,col_list_except_float]

            # NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。
            convDf = pd.get_dummies(convDf, dummy_na=True, columns=col_list_except_float)

            #  全ての特徴量
            allFeatureDf = convDf.join(float_conv_df)
            
        else:
            # float以外のカラムが存在しない場合
            allFeatureDf = float_conv_df

            
    ##### float変換がない場合 ####
    else:
        # NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。
        #  全ての特徴量
        allFeatureDf = pd.get_dummies(convDf, dummy_na=True, columns=col_list)

    ###
    #dispalyDf(allFeatureDf, True, 5,  "allFeatureDf")

    if train_Flg:
        # trainの場合
        # 標準化
#        print("標準化 = ",datetime.now())
#        allFeatureDf =  standardScalerDf(allFeatureDf)


#         # Ridge で特徴量選択を行う。
#         print("特徴量選択 = ",datetime.now())
#         retFeatureDf , retPklFile = getSelectorFeature(allFeatureDf,  train_target_Df, feature_num,
#                                                        feature_select_type, selector_step)

        retFeatureDf = allFeatureDf
        retPklFile = "train_feature_ConfirmedCases.pkl"
        
    else:
        # testの場合
        # カラムをtrainと合わせる
        print("カラム調整 = ",datetime.now())
        testFeatureDf = getAdjustTestColDf(allFeatureDf, train_col_list)
#         # 標準化
#         print("標準化 = ",datetime.now())
#         testFeatureDf =  standardScalerDf(testFeatureDf)
        
        retFeatureDf = testFeatureDf
        retPklFile = ""
        
    
    print("end = ",datetime.now())
    return retFeatureDf , retPklFile

# process

In [ ]:
# ファイル読み込み(train)
train_read_Df = pd.read_csv(read_train_path,
                            header=0,
                            encoding="utf-8",
                            sep='\n',
                            delimiter=',',
                            engine='python',
                            dtype=object)


In [ ]:
# ファイル読み込み(test)
test_read_Df = pd.read_csv(read_test_path,
                           header=0,
                           encoding="utf-8",
                           sep='\n',
                           delimiter=',',
                           engine='python',
                           dtype=object)


In [ ]:
######################################

In [ ]:
trainDf = train_read_Df.copy()
testDf = test_read_Df.copy()

In [ ]:
####### カラム名変更処理　#######
if COL_NAME_CHG_FLG:
    trainDf = trainDf.rename(columns={"Id": id_col_name})



In [ ]:
dispalyDf(trainDf, True, 5,  "trainDf")

In [ ]:
dispalyDf(testDf, True, 5,  "testDf")

In [ ]:
# カラムの差分
set(trainDf.columns.tolist()) - set(testDf.columns.tolist())

In [ ]:
# targetのユニーク値
trainDf[target_col_name].unique()

In [ ]:
#################

In [ ]:
# 独自処理
print(trainDf["Province/State"].nunique())
print(trainDf["Country/Region"].nunique())
print(trainDf["Lat"].nunique())
print(trainDf["Long"].nunique())

In [ ]:
print(testDf["Province/State"].nunique())
print(testDf["Country/Region"].nunique())
print(testDf["Lat"].nunique())
print(testDf["Long"].nunique())

In [ ]:
#################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Df = trainDf.loc[:,[target_col_name]]
trainDf = trainDf.loc[:,[col_name for col_name in trainDf.columns.tolist() if col_name not in [id_col_name,target_col_name]]]

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = testDf.loc[:,[id_col_name]]
testDf = testDf.loc[:,[col_name for col_name in testDf.columns.tolist() if col_name != id_col_name]]

In [ ]:
#### train のデータ前処理
train_new_Df, train_pkl_file = getBeforeProcessDf(
    trainDf, drop_col_name_list + ["Fatalities"],
    float_conv_col_name_list, float_nan_conv_dic,
    True , train_target_Df,
    RANDOM_STATE_VAL, FEATURE_NUM,
    feature_select_type,
    SELECTOR_STEP, None, other_conv_process)

In [ ]:
dispalyDf(train_new_Df, True, 5,  "train_new_Df")

In [ ]:
# 前処理が終わったtrainのpklを保存
with open(os.path.join(pkl_dir_path, train_pkl_file), 'wb') as DFfile:
    pickle.dump(train_new_Df, DFfile, protocol=2)


In [ ]:
#### test のデータ前処理
test_new_Df, _ = getBeforeProcessDf(
    testDf, drop_col_name_list,
    float_conv_col_name_list, float_nan_conv_dic,
    False , None,
    None, None,
    None,
    None, train_new_Df.columns.tolist(), other_conv_process)


In [ ]:
dispalyDf(test_new_Df, True, 5,  "test_new_Df")

In [ ]:
# データを増やす。不均衡調整。
ros = RandomOverSampler(random_state=0)
X_over,y_over = ros.fit_sample(train_new_Df, train_target_Df.values.ravel())
print(len(X_over))
print(len(y_over))

In [ ]:
train_new_Df = pd.DataFrame(X_over,dtype=np.float,columns=["Date"])
train_target_Df = pd.DataFrame(y_over,dtype=np.float,columns=[target_col_name])

In [ ]:
dispalyDf(train_new_Df, True, 5,  "train_new_Df")

In [ ]:
dispalyDf(train_target_Df, True, 5,  "train_target_Df")

In [ ]:
# モデル入手
def getModel(train_Df, y_train_Df, random_state_val, cv_val):
    print("cv_val = ",cv_val)
    model_def_list = [
        [
            "Ridge",
            Ridge(random_state=random_state_val, alpha=1.3)
        ],
        [
            "OLS",
            LinearRegression()
        ],
        [
            "SGDRegressor",
            SGDRegressor(random_state=random_state_val, alpha=0.3, max_iter=55000, eta0=0.0005)
        ]
    
    ]
    
    
    ret_model_list = []
    test_score_list = []
    
    
    for model_elem in model_def_list:
        print(" ==================== {} ====================".format(model_elem[0]))
        model = model_elem[1]
        start = time.time()
        model.fit(train_Df, y_train_Df.values.ravel())
        ret_model_list.append(model)

        cv_results = cross_validate(model,
                                                         train_Df,
                                                         y_train_Df.values.ravel(),      #1次元のARRAY
                                                         cv=cv_val,
                                                         scoring='r2')
#         print(cv_results)
#         print(cv_results["test_score"])
        print("test_score = ", np.mean(cv_results["test_score"]))
        test_score_list.append(np.mean(cv_results["test_score"]))

        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    
    print("----------------------------------------------------------------------------------------")
    # test_score が最大のアルゴリズムを適用する
    print(test_score_list)
    selected_index = np.argmax(test_score_list)
    print("selected_model : {}".format(model_def_list[selected_index][0]))
    return ret_model_list[selected_index]


In [ ]:
# モデル入手
model = getModel(train_new_Df, train_target_Df, RANDOM_STATE_VAL, CV_VAL)


In [ ]:
# 予測
resultDf =getPredictDf(model, test_new_Df, test_id_Df , target_col_name)


In [ ]:
dispalyDf(resultDf, True, 5,  "resultDf")

In [ ]:
# ディレクトリなければ作る
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
ouput_file_path = os.path.join(output_dir_path, output_file_name)
print(ouput_file_path)

In [ ]:
# ファイル出力
resultDf.to_csv(ouput_file_path,
               sep=",",
                header=True,
                index=False,
                mode="w",
                encoding="utf-8",
                line_terminator="\n"
               )


In [ ]:
########################################################################

In [ ]:
raise Exception("END♪")

In [ ]:
########################################################################